# Narnian DEMO
During this experience, we will face some classical tasks in signal processing to introduce the world of NARNIAN. This tutorial will help the user to understand some basic interactions that can happen among different Agents, presenting a framework that is general and open to customization. We will define the Environment, its Streams and the behaviors of its inhabitant by defining Finite State Machines and we will also visualize the interactions among those agents through an online viewer.

In the [first example](#One-signal,-one-repetition) we will show the case in which the Teacher shows the Student a single stream and, after this single repetition, asks him to repeat it. During the [second example](#Fixed-Playlist), the Teacher alternates two signals and finally ask the Student to repeat both of them. Finally, in the [last example](#Complete-Learning-Cycle) we will evaluate the simplest complete scenario in NARNIAN, where the Teacher asks the Student to both replicate the signals and to describe their own output. The lesson from the Teacher is repeated until the Student doesn't met a threshold on a given metric, set by the Teacher. Once the Student reaches the required level on the first signal, the Teacher moves to the second one.

### Install Dependencies (if you didn't already)

In [ ]:
%pip install -r -q requirements.txt

### Load Dependencies

In [1]:
import torch
from narnian.model import Model
from narnian.attributes import Attributes
from modules.networks import PredRNN
from modules.networks import GenCTBEInitStateBZeroInput, set_seed
from narnian.server import Server
from narnian.streams import Stream
from narnian.model import EmptyModel
from basic.basic_agent import BasicAgent
from basic.basic_streams import Sin, Square, CombSin
from basic.basic_environment import BasicEnvironment

### Define the Models used in the Demo

A **Narnian** model generally consists of a **generator** and a **predictor**. Here we will define the `DemoModel` as follows:
- `CTBE`: A Structured State Space Model exploiting Unitary matrices to produce perpetual oscillations (used as *Generator*);
- `PredRNN`: A simple rnn model processing the *Generator*'s output online to produce its descriptor (used as *Predictor*).

Along the model's definition, DemoModel includes also the `learn` method which, in this specific case, it is implemented as a common online GD training step.

In [2]:
class DemoModel(Model):

    def __init__(self, attributes: list[Attributes], lr: float = 0.0001, seed: int = -1, device: torch.device = torch.device("cpu")):
        """Creates a model composed of a generator and a predictor."""

        # getting shape info from attributes (it is needed to build the generator/predictor)
        assert len(attributes) == 2, "Only two attributes are supported/expected (about y and d)"
        u_shape = attributes[0].shape
        d_dim = attributes[1].shape.numel()
        y_dim = attributes[0].shape.numel()
        set_seed(seed)
        
        generator = GenCTBEInitStateBZeroInput(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=500, delta=0.1,
                                               local=False, cnu_memories=0)
        predictor = PredRNN(y_dim=y_dim, d_dim=d_dim, h_dim=10)

        # creating the model (superclass)
        super(DemoModel, self).__init__(generator, predictor, attributes, device=device)

        # extra stuff
        self.optim = torch.optim.SGD(list(self.generator.parameters()) + list(self.predictor.parameters()), lr=lr)
        self.loss_gen = torch.nn.functional.mse_loss
        self.loss_pred = torch.nn.functional.mse_loss

    def learn(self,
              y: torch.Tensor | None, yhat: torch.Tensor | None,
              d: torch.Tensor | None, dhat: torch.Tensor | None) \
            -> tuple[float, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        """Learn from different types of data (some of them could be None)."""

        # clean arguments, ensuring that what should be forced to None is actually forced to None
        _, y, yhat, d, dhat = super().learn(y, yhat, d, dhat)  # it seems unuseful, but IT MUST be called!

        # evaluating loss function
        loss = ((self.loss_gen(y, yhat) if yhat is not None else 0.) +
                (self.loss_pred(d, dhat) if dhat is not None else 0.))

        # learning
        self.optim.zero_grad()
        loss_as_float = loss.item()
        loss.backward()
        self.optim.step()

        return loss_as_float, y, yhat, d, dhat


## One signal, one repetition

In this first simple case, the Teacher shows 1000 samples of a target signal and immediately asks the Student to reproduce it.

We create the environment with name 'env' and we also set the title for visualization as 'Demo Signal Sandbox'.

In [3]:
# creating environment
env = BasicEnvironment("env", title="Demo Signal Sandbox")
device = torch.device("cpu")

Then we define a new stream which is a combination of 3 Sine waves with random frequencies.\
A stream is characterized by a name, its source (the enviroment in this case) and the stream itself.

In [4]:
env.add_stream(Stream.create(name="3sin", creator=env.name, stream=CombSin(f_cap=0.06, c_cap=0.8, delta=0.1, order=3)))

### Define the Environment

The Environment has 3 fundamental roles:
  1. to create its own streams and enable them to be experienced by agents;
  2. to ensure all the agents are aware of the existing streams;
  3. to ensure all the agents know each other.

In [5]:
# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

### Define the Teacher agent

We define the agent that will act the role of the Teacher in this demo.

Since each agent is characterized by a name, a model and its autority within the environment, we can now define the teacher agent as follows:
- Name: 'Teacher'
- Model: A dummy model
- Authority: 1 (the highest)

In [6]:
teacher = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)

Let's define the FSM of the teacher agent...

1\. Let the teacher agent know that the streams are now available. (init -> got_streams) _Rember that in the enviroment FSM we have enabled the streams_

In [7]:
teacher.add_transit("init", "got_streams", action="get_streams")

2\. Now the Teacher is aware of the streams but doesn't know the other agents. 

In [8]:
teacher.add_transit("got_streams", "got_agents", action="get_agents")

3\. The teacher, after knowing the other agents, is ready to record the streams flowing in the enviroment.

To identify the streams, we can use the `stream_hash` attribute, in this specific case the associated hash is "env:3sin"
We decide to record 1000 samples of the stream.

In [9]:
teacher.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":3sin", "steps": 1000})

The teacher, after recording the streams, start to find for agents to engage.
_(Since this action does not imply any change in the FSM, we add a state action with the state recording1)_

`min_auth` and the `max_auth` are set to 0.0, meaning that we are waiting for agent with autority strictly equal to 0

In [10]:
teacher.add_state_action("recording1", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})

4\. When students are found the teacher starts an engagement with them.

In [11]:
teacher.add_transit("recording1", "student_found", action="send_engagement")

5\. All the students which answer with a positive feedback are then engaged in the learning process.

In [12]:
teacher.add_transit("student_found", "student_engaged", action="got_engagement")

6\. The streams are shared with the students.

In [13]:
teacher.add_transit("student_engaged", "stream_shared", action="share_streams")

7\. The teacher asks the students to learn the function (in this case the sin).

We set the input-output/descriptor stream to the stream hash of the teacher.
We ask to learn for 1000 steps

In [14]:
teacher.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
                    args={"du_hash": teacher.name + ":recorded1", "yhat_hash": teacher.name + ":recorded1",
                          "dhat_hash": teacher.name + ":recorded1", "ask_steps": 1000})

8\. We end up the learning stage and we ask the students to generate the function.

In [15]:
teacher.add_transit("asked_learn", "done_learn", action="done_learn_gen")
teacher.add_transit("done_learn", "asked_gen", action="ask_gen",
                    args={"du_hash": teacher.name + ":recorded1", "dhat_hash": teacher.name + ":recorded1", "ask_steps": 1000})

9\. We end up the process and move to the end state.

In [16]:
teacher.add_transit("asked_gen", "finished", action="done_gen")

At this point we can add the Teacher to the environment.

In [17]:
env.add_agent(teacher)

### Define a student agent

The Student is nothing but a different instance of a NARNIAN Agent, so it will be defined similarly to the Teacher case. The main difference is that in this case we will use the DemoModel defined above and the Student's authority will be set to 0:
- Name: 'Student'
- Model: DemoModel
- Authority: 0 (the lowest)

In [18]:
student = BasicAgent("Student", model=DemoModel(attributes=env.shared_attributes, lr=0.005, device=device, seed=4), authority=0.0)

We now set its FSM...

1. the Student starts by acquiring the information of which streams are circulating in the enviroment;
2. the Student knows who are the other agents in the environment;
3. the Student is ready to engage with a Teacher (notice now that we are looking for `min_auth` and `max_auth` equal to 1);
4. after the engagement with a Teacher, the Student receives information about the streams available from that Teacher;
5. the Student go through the learning phase;
6. the Student generates back the signal. 

In [19]:
# creating student FSM
student.add_transit("init", "got_streams", action="get_streams")
student.add_transit("got_streams", "got_agents", action="get_agents")
student.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
student.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
student.add_transit("got_teacher_streams", "learning", action="do_learn_gen")
student.add_transit("got_teacher_streams", "generated", action="do_gen")
student.add_transit("learning", "got_teacher_streams", action="nop")
student.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(student)

Add a server to let the enviroment be accesible with a web app an d run the simulation.

In [20]:
Server(env=env, port=0)
env.run()

 * Serving Flask app 'narnian.server'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:53332
 * Running on http://192.168.1.14:53332
Press CTRL+C to quit
192.168.1.14 - - [21/Mar/2025 09:01:45] "GET / HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:45] "GET /index_dist.css HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:45] "GET /static/css/main.167a0901.css HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:45] "GET /static/js/main.771a64e2.js HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:46] "GET /get_env_name HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:46] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:46] "GET /get_play_pause_status HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:46] "GET /favicon.ico HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:47] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:01:47] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
192.168.1.14 - - 

[envir: env] >>> Clock step 0 <<<


192.168.1.14 - - [21/Mar/2025 09:01:52] "GET /get_play_pause_status HTTP/1.1" 200 -


[envir: env, state: init, act: enable_all_streams()] Enabling all streams
[envir: env] >>> Clock step 1 <<<
[envir: env, state: streams_enabled, act: send_streams_to_all()] Providing 1 streams to all agents
[agent: Teacher, state: init, act: get_streams(agent=None, streams={...})] Getting 1 streams from the environment
[agent: Student, state: init, act: get_streams(agent=None, streams={...})] Getting 1 streams from the environment
[envir: env] >>> Clock step 2 <<<
[envir: env, state: streams_sent, act: send_agents_to_all()] Sharing contacts of 2 agents with all the other agents
[agent: Teacher, state: got_streams, act: get_agents(agent=None, agents={...})] Getting contacts of 2 agents from the environment
[agent: Student, state: got_streams, act: get_agents(agent=None, agents={...})] Getting contacts of 2 agents from the environment
[envir: env] >>> Clock step 3 <<<


...

192.168.1.14 - - [21/Mar/2025 09:04:16] "GET /get_play_pause_status HTTP/1.1" 200 -


[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2932 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2933 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2934 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2935 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2936 <<<
[agent: Student, state: g

192.168.1.14 - - [21/Mar/2025 09:04:18] "GET /get_play_pause_status HTTP/1.1" 200 -


[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2973 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2974 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2975 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2976 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded1', u_hash=None, du_hash='Teacher:recorded1', steps=1000)] Generating signal
[envir: env] >>> Clock step 2977 <<<
[agent: Student, state: g

192.168.1.14 - - [21/Mar/2025 09:04:20] "GET /get_play_pause_status HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:04:20] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:04:20] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:04:20] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:04:20] "GET /get_list_of_streams?agent_name=Student HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:06:42] "GET /get_list_of_streams?agent_name=Student HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:06:42] "GET /get_console?agent_name=Student&last_only=true HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:06:42] "GET /get_console?agent_name=Student HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:06:42] "GET /get_behav?agent_name=Student HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:06:42] "GET /get_behav_status?agent_name=Student HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 

## Fixed Playlist

<a id='second_example'></a>

In this case, we are creating a Sandbox in which the Teacher has 2 different stream samples available for its Students.\
The lesson is structured as a playlist in which the Teacher provides the supervision, alternating the two signals.\
After 5 repetitions of each signal, the Student is asked to reproduce both of them.

### Adapt the Environment

We need to:
1. remove the Agents before we create the new ones;
2. add the new signal;
3. reset the state of the Environment to the Init state of its FSM.

In [21]:
env.remove_all_agents()
env.add_stream(Stream.create(name="square", creator=env.name, stream=Square(freq=0.06, ampl=0.5, phase=0.5, delta=0.1)))
env.behav.reset_state()

Successfully removed all agents!


### Create the new Teacher

We start as in the previous case but then we also record the second signal.

In [22]:
# creating teacher agent
teacher = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
teacher.add_transit("init", "got_streams", action="get_streams")
teacher.add_transit("got_streams", "got_agents", action="get_agents")
teacher.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":3sin", "steps": 1000})
teacher.add_transit("recording1", "recording2", action="record", args={"stream_hash": env.name + ":square", "steps": 1000})

#### Create the Playlist

Once all the streams have been recorded we can arrange the playlist, setting 5 repetitions supervised from the Teacher and a final request for generation (5+1).

In [23]:
teacher.add_transit("recording2", "playlist_ready", action="set_pref_streams",
                    args={"stream_hashes": [teacher.name + ":recorded1", teacher.name + ":recorded2"], "repeat": 5+1})

teacher.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})

Then we engage the student before we can start the lesson.

In [24]:
teacher.add_transit("playlist_ready", "student_found", action="send_engagement")
teacher.add_transit("student_found", "playlist_ready", action="nop")
teacher.add_transit("student_found", "student_engaged", action="got_engagement")
teacher.add_transit("student_engaged", "stream_shared", action="share_streams")

We ask the Student to learn to generate.

In [25]:
teacher.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
                    args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                          "ask_steps": 500})
teacher.add_transit("asked_learn", "done_learn", action="done_learn_gen")

#### Conditionally determine the next State

Go to the next Stream in the Playlist and check two options:
- if we are not yet in the last round of repetitions, go back to the `stream_shared` state and ask for learning;
- otherwise jump to the `ready_to_ask` state and ask for unsupervised generation.

In [26]:
teacher.behav.add_state_action("done_learn", action="next_pref_stream")
teacher.behav.add_transit("done_learn", "stream_shared", action="check_pref_stream", args={"what": "not_last_round"})
teacher.behav.add_transit("done_learn", "ready_to_ask", action="check_pref_stream", args={"what": "last_round"})

Coming out from this loop, we ask the Student to generate the signals without supervision.

In [27]:
teacher.behav.add_transit("ready_to_ask", "asked_gen", action="ask_gen",
                          args={"du_hash": "<playlist>",  "dhat_hash": "<playlist>", "ask_steps": 1000})
teacher.behav.add_transit("asked_gen", "done_gen", action="done_gen")

We need to add another check to loop inside the last repetition of the Playlist to play both songs and to go to the `finished` state after the last one.

In [28]:
teacher.behav.add_state_action("done_gen", action="next_pref_stream")
teacher.behav.add_transit("done_gen", "ready_to_ask", action="check_pref_stream", args={"what": "not_first"})
teacher.behav.add_transit("done_gen", "finished", action="check_pref_stream", args={"what": "first"})

env.add_agent(teacher)

### Create the new Student

As we have done [here](#Define-a-student-agent), we create a student with the same FSM as before and we add it to the environment.

In [29]:
student = BasicAgent("Student", model=DemoModel(attributes=env.shared_attributes, lr=0.005, device=device, seed=4), authority=0.0)
student.add_transit("init", "got_streams", action="get_streams")
student.add_transit("got_streams", "got_agents", action="get_agents")
student.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
student.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
student.add_transit("got_teacher_streams", "learning", action="do_learn_gen")
student.add_transit("got_teacher_streams", "generated", action="do_gen")
student.add_transit("learning", "got_teacher_streams", action="nop")
student.add_transit("generated", "got_teacher_streams", action="nop")

We add the agent to the environment and run it again.

In [31]:
env.add_agent(student)
Server(env=env, port=0)
env.run()

 * Serving Flask app 'narnian.server'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:53782
 * Running on http://192.168.1.14:53782
Press CTRL+C to quit
192.168.1.14 - - [21/Mar/2025 09:08:41] "GET / HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:41] "GET /index_dist.css HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:41] "GET /static/css/main.167a0901.css HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:41] "GET /static/js/main.771a64e2.js HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:42] "GET /get_env_name HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:42] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:42] "GET /get_play_pause_status HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:42] "GET /favicon.ico HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:43] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:08:43] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
192.168.1.14 - - 

[envir: env] >>> Clock step 0 <<<
[envir: env, state: init, act: enable_all_streams()] Enabling all streams
[envir: env] >>> Clock step 1 <<<
[envir: env, state: streams_enabled, act: send_streams_to_all()] Providing 2 streams to all agents
[agent: Teacher, state: init, act: get_streams(agent=None, streams={...})] Getting 2 streams from the environment


192.168.1.14 - - [21/Mar/2025 09:08:48] "GET /get_play_pause_status HTTP/1.1" 200 -


[agent: Student, state: init, act: get_streams(agent=None, streams={...})] Getting 2 streams from the environment
[envir: env] >>> Clock step 2 <<<
[envir: env, state: streams_sent, act: send_agents_to_all()] Sharing contacts of 2 agents with all the other agents
[agent: Teacher, state: got_streams, act: get_agents(agent=None, agents={...})] Getting contacts of 2 agents from the environment
[agent: Student, state: got_streams, act: get_agents(agent=None, agents={...})] Getting contacts of 2 agents from the environment
[envir: env] >>> Clock step 3 <<<
[agent: Teacher, state: got_agents, act: record(stream_hash='env:3sin', steps=1000)] Recording stream env:3sin
[envir: env] >>> Clock step 4 <<<
[agent: Teacher, state: got_agents, act: record(stream_hash='env:3sin', steps=1000)] Recording stream env:3sin
[envir: env] >>> Clock step 5 <<<
[agent: Teacher, state: got_agents, act: record(stream_hash='env:3sin', steps=1000)] Recording stream env:3sin
[envir: env] >>> Clock step 6 <<<
[agent:

...

192.168.1.14 - - [21/Mar/2025 09:16:21] "GET /get_play_pause_status HTTP/1.1" 200 -


[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 8982 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 8983 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 8984 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 8985 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 8986 <<<
[agent: Student, state: g

192.168.1.14 - - [21/Mar/2025 09:16:22] "GET /get_play_pause_status HTTP/1.1" 200 -


[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 9016 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 9017 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 9018 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 9019 <<<
[agent: Student, state: got_teacher_streams, act: gen(dhat_hash='Teacher:recorded2', u_hash=None, du_hash='Teacher:recorded2', steps=1000)] Generating signal
[envir: env] >>> Clock step 9020 <<<
[agent: Student, state: g

192.168.1.14 - - [21/Mar/2025 09:16:23] "GET /get_play_pause_status HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:16:23] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:16:23] "GET /get_list_of_streams?agent_name=env HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:16:23] "GET /get_list_of_streams?agent_name=Teacher HTTP/1.1" 200 -
192.168.1.14 - - [21/Mar/2025 09:16:23] "GET /get_list_of_streams?agent_name=Student HTTP/1.1" 200 -


## Complete Learning Cycle

This scenario encapsulates the simplest complete learning cycle within NARNIAN:
- the Teacher provides a lesson through recorded signals;
- the Student is tasked with both replicating the provided signal and describing its own output;
- continuous evaluation is performed until the Student’s performance meets a defined threshold;
- the process is sequential, ensuring that the Teacher moves to the next lesson (signal) only after the successful completion of previous one.

### Adapt the Environment

This time we will replace the 3Sin signal with a pure Sine wave, so we need to:
1. remove the Agents before we create the new ones;
2. remove the old signal and add the new one;
3. reset the state of the Environment to the Init state of its FSM.

In [37]:
env.remove_all_agents()
env.remove_stream(creator=env.name, name='3sin')
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))
env.behav.reset_state()

Successfully removed all agents!
Stream 3sin:env is unknown!! Maybe you were looking for one of these ['env:3sin', 'env:square', 'env:sin'].


### Define the teacher once again

In [33]:
teacher = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
teacher.add_transit("init", "got_streams", action="get_streams")
teacher.add_transit("got_streams", "got_agents", action="get_agents")
teacher.add_transit("got_agents", "recording1", action="record",
               args={"stream_hash": env.name + ":sin", "steps": 1000})
teacher.add_transit("recording1", "recording2", action="record",
               args={"stream_hash": env.name + ":square", "steps": 1000})
teacher.add_transit("recording2", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [teacher.name + ":recorded1", teacher.name + ":recorded2"], "repeat": 1})
teacher.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
teacher.add_transit("playlist_ready", "student_found", action="send_engagement")
teacher.add_transit("student_found", "playlist_ready", action="nop")
teacher.add_transit("student_found", "student_engaged", action="got_engagement")
teacher.add_transit("student_engaged", "stream_shared", action="share_streams")

#### Learning and Generation phase

The Teacher instructs the Student to learn (both generation and prediction) using the playlist data, and then to generate signals, each for 500 steps.

In [34]:
teacher.add_transit("stream_shared", "asked_learn", action="ask_learn_gen_and_pred",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                     "ask_steps": 1000})
teacher.add_transit("asked_learn", "done_learn", action="done_learn_gen_and_pred")
teacher.add_transit("done_learn", "asked_gen", action="ask_gen_and_pred",
               args={"du_hash": "<playlist>", "ask_steps": 1000})
teacher.add_transit("asked_gen", "done_gen", action="done_gen_and_pred")

#### Evaluation phase

The Teacher evaluates the generated signal by computing the Mean Squared Error (MSE) over 1000 steps. 
Based on the evaluation result:
- If MSE > 0.05, the process loops back to the shared stream state.
- If MSE <= 0.05, it moves to a "good" state.

In [ ]:
teacher.add_state_action("done_gen", action="eval",
                    args={"stream_hash": "<playlist>", "what": "y", "how": "mse", "steps": 1000})
teacher.add_transit("done_gen", "stream_shared", action="compare_eval", args={"cmp": ">", "thres": 0.05})
teacher.add_transit("done_gen", "good", action="compare_eval", args={"cmp": "<=", "thres": 0.05})

#### Playlist progression

When in the "good" state, the Teacher advances to the next preferred stream.
Depending on whether it is the first in the playlist or not, the process either repeats or finishes.

In [ ]:
teacher.add_state_action("good", action="next_pref_stream")
teacher.add_transit("good", "stream_shared", action="check_pref_stream", args={"what": "not_first"})
teacher.add_transit("good", "finished", action="check_pref_stream", args={"what": "first"})

Add the teacher to the environment.

In [ ]:
env.add_agent(teacher)

### Creating the Student

In [ ]:
student = BasicAgent("Student", model=DemoModel(attributes=env.shared_attributes, lr=0.001, device=device, seed=4),
                authority=0.0)
student.add_transit("init", "got_streams", action="get_streams")
student.add_transit("got_streams", "got_agents", action="get_agents")
student.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
student.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
student.add_transit("got_teacher_streams", "learning", action="do_learn_gen_and_pred")
student.add_transit("got_teacher_streams", "generated", action="do_gen_and_pred")
student.add_transit("learning", "got_teacher_streams", action="nop")
student.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(student)

In [ ]:
Server(env=env, port=0)
env.run()